# OpenAI Agents SDK - Customer Support System

It handles inquiries, creates tickets, processes payments, and maintains context across interactions.


In [1]:
%uv pip install openai-agents-sdk openai pydantic litellm --quiet


  × No solution found when resolving dependencies:
  ╰─▶ Because openai-agents-sdk was not found in the package registry and
      you require openai-agents-sdk, we can conclude that your requirements
      are unsatisfiable.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import datetime
from typing import Dict, Any, List, Optional
from pydantic import BaseModel, Field
from openai import OpenAI
from openai_agents_sdk import create_agent, run_agent

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("✅ Dependencies loaded successfully!")


In [ ]:
class Customer(BaseModel):
    customer_id: str = Field(..., description="Unique customer identifier")
    name: str = Field(..., min_length=1, max_length=100)
    email: str = Field(..., regex=r'^[^@]+@[^@]+\.[^@]+$')
    phone: Optional[str] = None
    tier: str = Field(default="standard", description="Customer tier: basic, standard, premium")
    created_at: datetime.datetime = Field(default_factory=datetime.datetime.now)

class SupportTicket(BaseModel):
    ticket_id: str = Field(..., description="Unique ticket identifier")
    customer_id: str
    title: str = Field(..., min_length=1, max_length=200)
    description: str = Field(..., min_length=10)
    category: str = Field(..., description="technical, billing, general")
    priority: str = Field(default="medium", description="low, medium, high, urgent")
    status: str = Field(default="open", description="open, in_progress, resolved, closed")
    assigned_agent: Optional[str] = None
    created_at: datetime.datetime = Field(default_factory=datetime.datetime.now)
    updated_at: datetime.datetime = Field(default_factory=datetime.datetime.now)

class ConversationContext(BaseModel):
    session_id: str
    customer: Customer
    current_agent: str
    conversation_history: List[Dict[str, str]] = Field(default_factory=list)
    active_ticket: Optional[SupportTicket] = None
    context_data: Dict[str, Any] = Field(default_factory=dict)
    last_interaction: datetime.datetime = Field(default_factory=datetime.datetime.now)

print("✅ Pydantic models defined")


In [ ]:
class ContextManager:
    def __init__(self):
        self.contexts: Dict[str, ConversationContext] = {}
        self.tickets: Dict[str, SupportTicket] = {}
        self.customers: Dict[str, Customer] = {}
    
    def get_or_create_context(self, customer_id: str, agent_name: str) -> ConversationContext:
        session_id = f"{customer_id}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        if customer_id not in self.customers:
            self.customers[customer_id] = Customer(
                customer_id=customer_id,
                name=f"Customer {customer_id}",
                email=f"customer{customer_id}@example.com"
            )
        
        context = ConversationContext(
            session_id=session_id,
            customer=self.customers[customer_id],
            current_agent=agent_name
        )
        
        self.contexts[session_id] = context
        return context
    
    def update_context(self, session_id: str, user_message: str, agent_response: str):
        if session_id in self.contexts:
            context = self.contexts[session_id]
            context.conversation_history.append({
                "timestamp": datetime.datetime.now().isoformat(),
                "user": user_message,
                "agent": agent_response
            })
            context.last_interaction = datetime.datetime.now()
    
    def create_ticket(self, customer_id: str, title: str, description: str, category: str) -> SupportTicket:
        ticket_id = f"TKT-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
        ticket = SupportTicket(
            ticket_id=ticket_id,
            customer_id=customer_id,
            title=title,
            description=description,
            category=category
        )
        self.tickets[ticket_id] = ticket
        return ticket

context_manager = ContextManager()
print("✅ Context management system initialized")


In [ ]:
class SupportTracer:
    def __init__(self):
        self.traces: List[Dict[str, Any]] = []
        self.current_trace: Optional[Dict[str, Any]] = None
    
    def start_trace(self, agent_name: str, customer_id: str, message: str):
        self.current_trace = {
            "trace_id": f"trace_{len(self.traces) + 1}",
            "agent_name": agent_name,
            "customer_id": customer_id,
            "user_message": message,
            "start_time": datetime.datetime.now(),
            "events": [],
            "tool_calls": [],
            "handoffs": [],
            "end_time": None,
            "duration": None,
            "response": None
        }
    
    def log_tool_call(self, tool_name: str, parameters: Dict, result: str):
        if self.current_trace:
            self.current_trace["tool_calls"].append({
                "timestamp": datetime.datetime.now(),
                "tool_name": tool_name,
                "parameters": parameters,
                "result": result[:200] + "..." if len(result) > 200 else result
            })
    
    def log_handoff(self, from_agent: str, to_agent: str, reason: str):
        if self.current_trace:
            self.current_trace["handoffs"].append({
                "timestamp": datetime.datetime.now(),
                "from_agent": from_agent,
                "to_agent": to_agent,
                "reason": reason
            })
    
    def log_event(self, event_type: str, data: Any):
        if self.current_trace:
            self.current_trace["events"].append({
                "timestamp": datetime.datetime.now(),
                "event_type": event_type,
                "data": data
            })
    
    def end_trace(self, response: str):
        if self.current_trace:
            self.current_trace["end_time"] = datetime.datetime.now()
            self.current_trace["duration"] = (
                self.current_trace["end_time"] - self.current_trace["start_time"]
            ).total_seconds()
            self.current_trace["response"] = response
            self.traces.append(self.current_trace.copy())
            self.current_trace = None
    
    def get_analytics(self) -> Dict[str, Any]:
        if not self.traces:
            return {"message": "No traces available"}
        
        return {
            "total_interactions": len(self.traces),
            "avg_duration": sum(t.get("duration", 0) for t in self.traces) / len(self.traces),
            "total_tool_calls": sum(len(t.get("tool_calls", [])) for t in self.traces),
            "total_handoffs": sum(len(t.get("handoffs", [])) for t in self.traces),
            "agent_usage": {agent: sum(1 for t in self.traces if t.get("agent_name") == agent) 
                          for agent in set(t.get("agent_name", "") for t in self.traces)}
        }

tracer = SupportTracer()
print("✅ Tracing system initialized")


In [ ]:
class ContentGuardrails:
    def __init__(self):
        self.blocked_words = ['hack', 'exploit', 'illegal', 'fraud', 'scam']
        self.sensitive_topics = ['password', 'credit card', 'ssn', 'social security']
        self.profanity = ['inappropriate', 'offensive']
    
    def check_content(self, text: str) -> Dict[str, Any]:
        text_lower = text.lower()
        
        blocked_found = [word for word in self.blocked_words if word in text_lower]
        sensitive_found = [topic for topic in self.sensitive_topics if topic in text_lower]
        profanity_found = [word for word in self.profanity if word in text_lower]
        
        is_safe = not (blocked_found or profanity_found)
        has_sensitive = bool(sensitive_found)
        
        return {
            "is_safe": is_safe,
            "has_sensitive_data": has_sensitive,
            "blocked_words": blocked_found,
            "sensitive_topics": sensitive_found,
            "profanity": profanity_found,
            "action": "block" if not is_safe else "warn" if has_sensitive else "allow"
        }
    
    def filter_response(self, response: str) -> str:
        check_result = self.check_content(response)
        
        if not check_result["is_safe"]:
            return "I apologize, but I cannot provide information on that topic due to our safety guidelines."
        
        if check_result["has_sensitive_data"]:
            return response + "\n\n⚠️ Please ensure you're in a secure environment when discussing sensitive information."
        
        return response

guardrails = ContentGuardrails()
print("✅ Guardrails system initialized")


In [ ]:
def search_knowledge_base(query: str, category: str = "general") -> str:
    """Search the knowledge base for relevant information"""
    tracer.log_tool_call("search_knowledge_base", {"query": query, "category": category}, "Knowledge search completed")
    
    kb_data = {
        "technical": {
            "login issues": "Try clearing browser cache, check credentials, or reset password",
            "slow performance": "Check internet connection, close unnecessary tabs, clear cache",
            "error messages": "Note the exact error message and steps to reproduce"
        },
        "billing": {
            "payment failed": "Check card details, ensure sufficient funds, contact bank if needed",
            "refund request": "Refunds processed within 5-7 business days to original payment method",
            "subscription": "Manage subscriptions in account settings or contact billing team"
        },
        "general": {
            "account": "Account related questions - check profile settings",
            "features": "Feature requests can be submitted through feedback form",
            "contact": "Multiple contact options available - chat, email, phone"
        }
    }
    
    results = []
    search_category = kb_data.get(category, kb_data["general"])
    
    for key, value in search_category.items():
        if any(word in key.lower() for word in query.lower().split()):
            results.append(f"**{key.title()}**: {value}")
    
    if not results:
        results = ["No specific matches found. Please provide more details about your issue."]
    
    return "\n".join(results)

def create_support_ticket(customer_id: str, title: str, description: str, category: str) -> str:
    """Create a new support ticket"""
    try:
        ticket = context_manager.create_ticket(customer_id, title, description, category)
        tracer.log_tool_call("create_support_ticket", 
                           {"customer_id": customer_id, "category": category}, 
                           f"Ticket {ticket.ticket_id} created")
        
        return json.dumps({
            "ticket_id": ticket.ticket_id,
            "status": "created",
            "message": f"Support ticket {ticket.ticket_id} has been created successfully."
        })
    except Exception as e:
        return json.dumps({"error": f"Failed to create ticket: {str(e)}"})

def process_payment(customer_id: str, amount: float, payment_method: str) -> str:
    """Process a payment (mock implementation)"""
    tracer.log_tool_call("process_payment", 
                       {"customer_id": customer_id, "amount": amount}, 
                       "Payment processed")
    
    if amount <= 0:
        return json.dumps({"error": "Invalid payment amount"})
    
    payment_id = f"PAY-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    return json.dumps({
        "payment_id": payment_id,
        "status": "completed",
        "amount": amount,
        "method": payment_method,
        "message": f"Payment of ${amount} processed successfully."
    })

def handoff_to_technical(reason: str, context: str, customer_id: str) -> str:
    """Handoff conversation to technical support"""
    tracer.log_handoff("CustomerService", "TechnicalSupport", reason)
    
    handoff_data = {
        "handoff_id": f"HO-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}",
        "from_agent": "CustomerService",
        "to_agent": "TechnicalSupport", 
        "reason": reason,
        "context": context,
        "customer_id": customer_id,
        "timestamp": datetime.datetime.now().isoformat()
    }
    
    return json.dumps({
        "status": "handoff_completed",
        "message": f"Transferring you to technical support. Reason: {reason}",
        "handoff_id": handoff_data["handoff_id"]
    })

def handoff_to_billing(reason: str, context: str, customer_id: str) -> str:
    """Handoff conversation to billing department"""
    tracer.log_handoff("CustomerService", "BillingSupport", reason)
    
    return json.dumps({
        "status": "handoff_completed", 
        "message": f"Transferring you to billing department. Reason: {reason}",
        "handoff_id": f"HO-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
    })

print("✅ Tool functions defined")


In [ ]:
knowledge_search_tool = {
    "type": "function",
    "function": {
        "name": "search_knowledge_base",
        "description": "Search knowledge base for solutions and information",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "category": {"type": "string", "enum": ["technical", "billing", "general"], "default": "general"}
            },
            "required": ["query"]
        }
    }
}

ticket_creation_tool = {
    "type": "function", 
    "function": {
        "name": "create_support_ticket",
        "description": "Create a new support ticket",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {"type": "string", "description": "Customer ID"},
                "title": {"type": "string", "description": "Ticket title"},
                "description": {"type": "string", "description": "Detailed description"},
                "category": {"type": "string", "enum": ["technical", "billing", "general"]}
            },
            "required": ["customer_id", "title", "description", "category"]
        }
    }
}

payment_tool = {
    "type": "function",
    "function": {
        "name": "process_payment", 
        "description": "Process a customer payment",
        "parameters": {
            "type": "object",
            "properties": {
                "customer_id": {"type": "string", "description": "Customer ID"},
                "amount": {"type": "number", "description": "Payment amount"},
                "payment_method": {"type": "string", "description": "Payment method"}
            },
            "required": ["customer_id", "amount", "payment_method"]
        }
    }
}

technical_handoff_tool = {
    "type": "function",
    "function": {
        "name": "handoff_to_technical",
        "description": "Transfer conversation to technical support",
        "parameters": {
            "type": "object",
            "properties": {
                "reason": {"type": "string", "description": "Reason for handoff"},
                "context": {"type": "string", "description": "Context summary"},
                "customer_id": {"type": "string", "description": "Customer ID"}
            },
            "required": ["reason", "context", "customer_id"]
        }
    }
}

billing_handoff_tool = {
    "type": "function",
    "function": {
        "name": "handoff_to_billing",
        "description": "Transfer conversation to billing department",
        "parameters": {
            "type": "object",
            "properties": {
                "reason": {"type": "string", "description": "Reason for handoff"},
                "context": {"type": "string", "description": "Context summary"},
                "customer_id": {"type": "string", "description": "Customer ID"}
            },
            "required": ["reason", "context", "customer_id"]
        }
    }
}

print("✅ Tool definitions created")


In [ ]:
customer_service_agent = create_agent(
    name="CustomerService",
    instructions="""You are a friendly customer service representative. Your role is to:
    
    1. Greet customers warmly and identify their needs
    2. Search the knowledge base for solutions to common issues
    3. Create support tickets for complex issues
    4. Transfer to technical support for technical problems
    5. Transfer to billing for payment and subscription issues
    
    Always be helpful, professional, and empathetic. If you cannot resolve an issue,
    create a ticket or transfer to the appropriate specialized team.""",
    model="gpt-4o-mini",
    tools=[knowledge_search_tool, ticket_creation_tool, technical_handoff_tool, billing_handoff_tool],
    client=client
)

technical_support_agent = create_agent(
    name="TechnicalSupport", 
    instructions="""You are a technical support specialist. Your expertise includes:
    
    1. Diagnosing technical problems and providing step-by-step solutions
    2. Searching technical knowledge base for detailed troubleshooting
    3. Creating technical tickets for complex issues requiring engineering
    4. Providing clear, actionable instructions
    
    Always ask clarifying questions to understand the exact issue and provide
    detailed technical solutions. Be patient and thorough in your explanations.""",
    model="gpt-4o-mini",
    tools=[knowledge_search_tool, ticket_creation_tool],
    client=client
)

billing_support_agent = create_agent(
    name="BillingSupport",
    instructions="""You are a billing specialist. Your responsibilities include:
    
    1. Handling payment processing and billing inquiries
    2. Processing refunds and subscription changes
    3. Searching billing knowledge base for policy information
    4. Creating billing tickets for account-specific issues
    
    Always verify customer identity before processing financial transactions.
    Be clear about billing policies and procedures.""",
    model="gpt-4o-mini", 
    tools=[knowledge_search_tool, payment_tool, ticket_creation_tool],
    client=client
)

print("✅ Specialized agents created:")
print(f"  - {customer_service_agent.name}")
print(f"  - {technical_support_agent.name}")
print(f"  - {billing_support_agent.name}")


In [ ]:
def run_support_interaction(agent, customer_id: str, message: str) -> str:
    """Run a complete support interaction with tracing and context management"""
    
    # Start tracing
    tracer.start_trace(agent.name, customer_id, message)
    
    # Check content safety
    safety_check = guardrails.check_content(message)
    if not safety_check["is_safe"]:
        tracer.log_event("content_blocked", safety_check)
        tracer.end_trace("Content blocked by safety filters")
        return "I'm sorry, but I cannot process that request due to our content policy."
    
    # Create or get context
    context = context_manager.get_or_create_context(customer_id, agent.name)
    tracer.log_event("context_created", {"session_id": context.session_id})
    
    # Define tool functions
    tool_functions = {
        "search_knowledge_base": search_knowledge_base,
        "create_support_ticket": create_support_ticket,
        "process_payment": process_payment,
        "handoff_to_technical": handoff_to_technical,
        "handoff_to_billing": handoff_to_billing
    }
    
    try:
        # Run the agent
        response = run_agent(agent, message, tool_functions=tool_functions)
        
        # Apply content filtering to response
        filtered_response = guardrails.filter_response(response)
        
        # Update context
        context_manager.update_context(context.session_id, message, filtered_response)
        
        # End tracing
        tracer.end_trace(filtered_response)
        
        return filtered_response
        
    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        tracer.log_event("error", {"error": str(e)})
        tracer.end_trace(error_message)
        return error_message

print("✅ Support interaction function ready")


In [ ]:
customer_message = "Hi, I'm having trouble logging into my account. Can you help?"
customer_id = "CUST001"

print("🔵 Customer Service Interaction")
print(f"Customer ({customer_id}): {customer_message}")
print()

response = run_support_interaction(customer_service_agent, customer_id, customer_message)
print(f"Agent Response: {response}")
print("\n" + "="*60)


In [ ]:
technical_message = "My application keeps crashing when I try to export large datasets. It worked fine yesterday."
customer_id = "CUST002"

print("🔴 Technical Support Needed")
print(f"Customer ({customer_id}): {technical_message}")
print()

# First, customer service handles the inquiry
response = run_support_interaction(customer_service_agent, customer_id, technical_message)
print(f"Customer Service: {response}")
print()

# If handoff occurred, continue with technical support
follow_up = "I need detailed troubleshooting steps for the export crash issue."
print(f"Customer: {follow_up}")
tech_response = run_support_interaction(technical_support_agent, customer_id, follow_up)
print(f"Technical Support: {tech_response}")
print("\n" + "="*60)


In [ ]:
billing_message = "My payment failed and I need to update my subscription. Can you help me process a new payment?"
customer_id = "CUST003"

print("🟡 Billing Support Interaction")
print(f"Customer ({customer_id}): {billing_message}")
print()

# Customer service handles initial inquiry
response = run_support_interaction(customer_service_agent, customer_id, billing_message)
print(f"Customer Service: {response}")
print()

# Continue with billing specialist
payment_request = "I want to pay $99.99 with my credit card for the premium subscription."
print(f"Customer: {payment_request}")
billing_response = run_support_interaction(billing_support_agent, customer_id, payment_request)
print(f"Billing Support: {billing_response}")
print("\n" + "="*60)


In [ ]:
unsafe_message = "Can you help me hack into my competitor's system?"
customer_id = "CUST004"

print("🚫 Content Safety Test")
print(f"Customer ({customer_id}): {unsafe_message}")
print()

response = run_support_interaction(customer_service_agent, customer_id, unsafe_message)
print(f"Agent Response: {response}")
print()

# Test sensitive content warning
sensitive_message = "I forgot my password and need help with my credit card information."
print(f"Customer: {sensitive_message}")
sensitive_response = run_support_interaction(customer_service_agent, customer_id, sensitive_message)
print(f"Agent Response: {sensitive_response}")
print("\n" + "="*60)


In [ ]:
print("📊 SYSTEM ANALYTICS")
print("="*50)

# Tracing analytics
analytics = tracer.get_analytics()
print("🔍 Tracing Analytics:")
for key, value in analytics.items():
    print(f"  {key}: {value}")
print()

# Context analytics
print("📝 Context Analytics:")
print(f"  Total customers: {len(context_manager.customers)}")
print(f"  Active sessions: {len(context_manager.contexts)}")
print(f"  Support tickets: {len(context_manager.tickets)}")
print()

# Show recent tickets
print("🎫 Recent Support Tickets:")
for ticket_id, ticket in list(context_manager.tickets.items())[-3:]:
    print(f"  {ticket_id}: {ticket.title} ({ticket.category}) - {ticket.status}")
print()

# Show customer information
print("👥 Customer Information:")
for customer_id, customer in context_manager.customers.items():
    print(f"  {customer_id}: {customer.name} ({customer.email}) - {customer.tier}")

print("\n" + "="*60)


In [ ]:
try:
    import litellm
    
    # Example of using different LLM providers with LiteLLM
    # This would allow switching between OpenAI, Anthropic, Google, etc.
    
    def create_multi_provider_agent(provider: str = "openai"):
        """Create agent with different LLM providers using LiteLLM"""
        
        # Configure different providers
        provider_configs = {
            "openai": {"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY")},
            "anthropic": {"model": "claude-3-sonnet", "api_key": os.getenv("ANTHROPIC_API_KEY")},
            "google": {"model": "gemini-pro", "api_key": os.getenv("GOOGLE_API_KEY")}
        }
        
        if provider in provider_configs:
            config = provider_configs[provider]
            print(f"✅ LiteLLM configured for {provider} with model {config['model']}")
            
            # Note: In actual implementation, you would configure the OpenAI Agents SDK
            # to use LiteLLM as the client for multi-provider support
            
            return f"Agent configured with {provider} provider"
        else:
            return f"Provider {provider} not supported"
    
    # Demo different providers
    print("🌐 LiteLLM Multi-Provider Support:")
    for provider in ["openai", "anthropic", "google"]:
        result = create_multi_provider_agent(provider)
        print(f"  {result}")
    
    print("\n💡 LiteLLM enables using multiple LLM providers with the same codebase")
    print("   This allows for cost optimization, redundancy, and model comparison")
    
except ImportError:
    print("⚠️  LiteLLM not available. Install with: pip install litellm")
    print("   LiteLLM provides unified interface for multiple LLM providers")
